# anglE

In [2]:
# Import all necessary libraries
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
#import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras import utils
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import bert
import os
import numpy as np
import re
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [3]:
colnames=['image_path', 'text', 'food']
train = pd.read_csv('train_titles.csv', names=colnames, header=None, sep = ',', index_col=['image_path'])
test = pd.read_csv('test_titles.csv', names=colnames, header=None, sep = ',', index_col=['image_path'])

In [4]:
test = test.sort_values('image_path')
train = train.sort_values('image_path')

In [5]:
train.head()

,text,food
image_path,,
apple_pie_0.jpg,Apple pie - Wikipedia,apple_pie
apple_pie_100.jpg,Glazed Apple Pie Squares Recipe | Taste of Home,apple_pie
apple_pie_101.jpg,Mock Apple Pie Recipe - Allrecipes.com,apple_pie
apple_pie_102.jpg,Crock-Pot Ladies Crock-Pot Apple Pie Moonshine,apple_pie
apple_pie_104.jpg,All-Star Apple Pie Recipe | Taste of Home,apple_pie


In [6]:
print("train samples:",train.shape[0])
print("test samples:",test.shape[0])

train samples: 67972
test samples: 22716


In [7]:
# Cleaning text function

def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    sentence = sentence.lower()

    return sentence

def remove_tags(text):
    return TAG_RE.sub('', text)

TAG_RE = re.compile(r'<[^>]+>')
vec_preprocess_text = np.vectorize(preprocess_text)

In [8]:
# Check number of classes
nClasses = train.food.nunique()

In [9]:
encoder = LabelEncoder()
processed_train = vec_preprocess_text(train.text.values)
processed_test = vec_preprocess_text(test.text.values)


encoded_labels_train = encoder.fit_transform(train.food.values)
labels_train = utils.to_categorical(encoded_labels_train, nClasses)

encoded_labels_test = encoder.fit_transform(test.food.values)
labels_test = utils.to_categorical(encoded_labels_test, nClasses)

print("Processed text sample:", processed_train[0])
print("Shape of train labels:", labels_train.shape)

Processed text sample: apple pie wikipedia
Shape of train labels: (67972, 101)


In [10]:
processed_train[0] 

'apple pie wikipedia'

In [11]:
from angle_emb import AnglE

angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()


/home/ktg3317/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
train_text = angle.encode(processed_train[0], to_numpy=True)
for i in range(1,67988):
    if i%5000==0:
        print(i/5000)
    vec = angle.encode(processed_train[i], to_numpy=True)
    train_text = np.concatentate((train_text,vec),axis=0)
np.load('text_train',train_text)

In [12]:
angle.encode(processed_train[1], to_numpy=True)

array([[-0.8674534 ,  0.28147906,  0.12365077, ..., -0.8258026 ,
         0.8993709 ,  0.00467708]], dtype=float32)

In [13]:
train_data = np.load('text_train.npy')

In [14]:
test_data = np.load('text_test.npy')

In [54]:
# Classification Model
input = layers.Input(shape=(1024,), dtype=tf.int32,
                                       name="input_data")

X = layers.Dense(512, activation="relu")(input)
X = layers.Dropout(0.5)(X)
X = layers.Dense(256, activation="relu")(X)
X = layers.Dropout(0.5)(X)
output = layers.Dense(nClasses, activation = 'softmax')(X)

model = tf.keras.models.Model(input, output)

In [62]:
opt = optimizers.Adam(lr=.001)

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

In [63]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 1024)]            0         
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 101)               25957     
                                                                 
Total params: 682,085
Trainable params: 682,085
Non-trainab

In [64]:
es = callbacks.EarlyStopping(monitor='accuracy', patience=2, restore_best_weights=True)

In [65]:
csv_logger = callbacks.CSVLogger('BERT_LSTM/BERT_LSTM.log')
es = callbacks.EarlyStopping(patience = 3, restore_best_weights=True)

In [66]:
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='accuracy', factor=0.1, patience=1, min_lr=0.00001)

In [67]:
history = model.fit(x=train_data, 
          y=labels_train,
          epochs = 20,
          batch_size = 512,
          callbacks = [csv_logger, reduce_lr])

Epoch 1/20
133/133 [==============================] - 2s 5ms/step - loss: 0.3826 - accuracy: 0.9036 - lr: 0.0010
Epoch 2/20
133/133 [==============================] - 1s 4ms/step - loss: 0.3651 - accuracy: 0.9054 - lr: 0.0010
Epoch 3/20
133/133 [==============================] - 0s 4ms/step - loss: 0.3590 - accuracy: 0.9074 - lr: 0.0010
Epoch 4/20
133/133 [==============================] - 0s 4ms/step - loss: 0.3440 - accuracy: 0.9110 - lr: 0.0010
Epoch 5/20
133/133 [==============================] - 1s 4ms/step - loss: 0.3321 - accuracy: 0.9146 - lr: 0.0010
Epoch 6/20
133/133 [==============================] - 1s 4ms/step - loss: 0.3277 - accuracy: 0.9143 - lr: 0.0010
Epoch 7/20
133/133 [==============================] - 1s 4ms/step - loss: 0.2961 - accuracy: 0.9225 - lr: 1.0000e-04
Epoch 8/20
133/133 [==============================] - 1s 4ms/step - loss: 0.2889 - accuracy: 0.9247 - lr: 1.0000e-04
Epoch 9/20
133/133 [==============================] - 1s 4ms/step - loss: 0.2818 - accur

In [68]:
model.evaluate(test_data,
               labels_test, 
               batch_size = 512)

45/45 [==============================] - 0s 2ms/step - loss: 0.8984 - accuracy: 0.8502


[0.8984338045120239, 0.8502377271652222]